<a href="https://colab.research.google.com/github/yclipse/RTS_mapping/blob/main/eeify_and_host_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_unet_collection
from keras_unet_collection import models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 8.7 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=NNCMxD6I9PX4zbYUZx3wDM9N27eL3YvPH0Yy2MEXJRU&tc=LNjk4F1QMwDWCttfM9NZVcYhSr2HeeuAhLevHSgM_XQ&cc=PvedWa35s_8ScwOEYLBYJTfYj-z89frm6_4ftlC0G94

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VNGc2pUpDAozf8Tu0l3YLbJZSrlh_VbW7InHTv14GfgEibPn7oSiKg

Successfully saved authorization token.


# Load model

In [ ]:
import json
with open('/content/drive/MyDrive/RTS_models/MODEL_V2_UNET3+_MAXAR_192x192/20230601-161139/params.json', 'r') as f:
  PARAMS = json.load(f)

model = models.unet_3plus_2d(input_size=PARAMS['model']['input_size'],
                  n_labels=2,
                  filter_num_down=PARAMS['model']['filter_num'],
                  filter_num_skip='auto',
                  filter_num_aggregate='auto',
                  stack_num_down=PARAMS['model']['stack_num_down'],
                  stack_num_up=PARAMS['model']['stack_num_up'],
                  activation=PARAMS['model']['activation'],
                  output_activation=PARAMS['model']['out_activ'],
                  batch_norm=PARAMS['model']['batch_norm'],
                  pool=PARAMS['model']['pooling'],
                  unpool=PARAMS['model']['unpool'],
                  deep_supervision=PARAMS['model']['deep_supervision'],
                  backbone=PARAMS['model']['backbone'],
                  weights=None,
                  freeze_backbone=PARAMS['model']['freeze_backbone'],
                  freeze_batch_norm=PARAMS['model']['freeze_bn'],
                  name='unet3plus')

# Load trained weights
model_path = '/content/drive/MyDrive/RTS_models/MODEL_V2_UNET3+_MAXAR_192x192/20230601-161139/cp-0109-valiou0.711.ckpt'
model.load_weights(model_path)

Automated hyper-parameter determination is applied with the following details:
----------
	Number of convolution filters after each full-scale skip connection: filter_num_skip = [32, 32, 32, 32]
	Number of channels of full-scale aggregated feature maps: filter_num_aggregate = 160
----------
deep_supervision = True
names of output tensors are listed as follows ("sup0" is the shallowest supervision layer;
"final" is the final output layer):

	unet3plus_output_sup0_activation
	unet3plus_output_sup1_activation
	unet3plus_output_sup2_activation
	unet3plus_output_sup3_activation
	unet3plus_output_final_activation


In [ ]:
import tensorflow as tf

adam = tf.keras.optimizers.Adam(learning_rate=PARAMS['train']['learning_rate'],
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=PARAMS['train']['adam_epsilon'],
                  amsgrad=PARAMS['train']['amsgrad'],
                  name='Adam')

# sgd = tf.keras.optimizers.SGD(learning_rate=PARAMS['train']['learning_rate'], momentum=0.9)

loss = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True,
    alpha=PARAMS['train']['focal_loss_alpha'],    #In practice α may be set by inverse class frequency or treated as a hyperparameter to set by cross validation
    gamma=2,    #dont change
    from_logits=False,
    label_smoothing=PARAMS['train']['label_smoothing'],
    axis=-1,
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_focal_crossentropy'
)

iou = [tf.keras.metrics.OneHotMeanIoU(num_classes=2, name='iou')]

model.compile(loss=loss,
       optimizer=adam,
       metrics=iou,
       )

# EEify the model

In [ ]:
import os
MODEL_DIR = '/content/drive/MyDrive/RTS_models/Deployed_Models'
MODEL_NAME = 'Unet3_MAXAR192_20230601_161139' #only underscores
model.save(os.path.join(MODEL_DIR, MODEL_NAME+'.tf'), save_format='tf')

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(os.path.join(MODEL_DIR, MODEL_NAME+'.tf'), 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "output"}) + "'"
print(input_dict)
print(output_dict)

'{"serving_default_input_2:0": "array"}'
'{"StatefulPartitionedCall:0": "output"}'


In [ ]:
# Put the EEified model next to the trained model directory.
EEIFIED_DIR = os.path.join(MODEL_DIR, MODEL_NAME+'_EEify')
PROJECT = 'abruptthawmapping'
REGION = 'us-east4'
VERSION_NAME = 'v3'
STAGING_BUCKET = 'gs://abrupt_thaw'

!gcloud config set project {PROJECT}
# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {os.path.join(MODEL_DIR, MODEL_NAME+'.tf')} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Updated property [core/project].
Successfully saved project id
Success: model at '/content/drive/MyDrive/RTS_models/Deployed_Models/Unet3_MAXAR192_20230601_161139_EEify' is ready to be hosted in AI Platform.


In [ ]:
%%writefile config.yaml
autoScaling:
  minNodes: 10

Overwriting config.yaml


In [ ]:
!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

Using endpoint [https://us-east4-ml.googleapis.com/]
Created ai platform model [projects/abruptthawmapping/models/Unet3_MAXAR192_20230601_161139].


In [ ]:

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --region {REGION} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --staging-bucket {STAGING_BUCKET} \
  --framework "TENSORFLOW" \
  --runtime-version=2.3 \
  --python-version=3.7

Using endpoint [https://us-east4-ml.googleapis.com/]


In [ ]:
# Load the trained model and use it for prediction.  If you specified a region
# other than the default (us-central1) at model creation, specify it here.
model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME,
    version=VERSION_NAME,
    region=REGION,
    # Can be anything, but don't make it too big.
    inputTileSize=[192, 192],
    inputOverlapSize=[32,32],
    # Keep this the same as your training data.
    proj=ee.Projection('EPSG:3413').atScale(2),
    fixInputProj=True,
    # Note the names here need to match what you specified in the
    # output dictionary you passed to the EEifier.
    outputBands={'prediction': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

print(model.getInfo())

{'type': 'Model.fromAiPlatformPredictor', 'projectName': 'abruptthawmapping', 'projectId': 'abruptthawmapping', 'modelName': 'Unet3plus_MAXAR192_20230601-161139.tf', 'version': 'v3', 'region': 'us-east4', 'inputProperties': [], 'inputTypeOverride': {}, 'inputShapes': {}, 'proj': {'type': 'Projection', 'crs': 'EPSG:3413', 'transform': [2, 0, 0, 0, 2, 0]}, 'fixInputProj': True, 'inputTileSize': [192, 192], 'inputOverlapSize': [32, 32], 'outputTileSize': [192, 192], 'outputBands': {'prediction': {'type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 1}}}, 'outputProperties': {}, 'outputMultiplier': 1}
